In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik
# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1700
RECS = 1384 * 4

X = np.zeros([RECS, LENGTH, 257], np.float32)
Y = np.zeros([RECS, LENGTH, 20], np.float32)
counter = 0

for i in range(4):
    Xpart = np.load("datasets/clarin-long/data/clarin-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

Xpart.shape

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()

valid = np.random.random(X.shape[0]) > 0.9

/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


5523 5536


In [2]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

from keras.constraints import max_norm
from keras.regularizers import L1L2

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

class StopOnConvergence(Callback):
    def __init__(self, max_repetitions=10):
        super().__init__()
        self.max_repetitions = max_repetitions

    def on_train_begin(self, logs=None):
        self.repetitions = 0
        self.last_loss = np.inf

    def on_epoch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('val_loss')
        if loss is not None:
            if loss > self.last_loss:
                self.repetitions += 1
            else:
                self.last_loss = loss
                self.repetitions = 0
            if self.repetitions > self.max_repetitions:
                self.model.stop_training = True

            
logger = MyCallback("./training-bigger-log.txt")
NFEATS = 257

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    dense1 = LeakyReLU(0.01)(Dense(1024, kernel_regularizer=L1L2(l1=1e-6))(feature_input))
    dense2 = LeakyReLU(0.01)(Dense(1024, kernel_regularizer=L1L2(l1=1e-6))(dense1))
    dense3 = LeakyReLU(0.01)(Dense(1024, kernel_regularizer=L1L2(l1=1e-6))(dense2))
    dense4 = LeakyReLU(0.01)(Dense(1024, kernel_regularizer=L1L2(l1=1e-6))(dense3))
    dense5 = LeakyReLU(0.01)(Dense(1024, kernel_regularizer=L1L2(l1=1e-6))(dense4))
    dense6 = Dense(20)(dense5)
    model = Model(feature_input, dense6)
    model.summary()
    return model

if __name__=='__main__':
    model = mk_model(Y.shape[1])
    model.compile(loss='mse', optimizer=Adam(clipnorm=1.))
    model.fit(X[~valid], Y[~valid], epochs = 100,
         batch_size = 8, callbacks=[mc, logger, StopOnConvergence(4)], validation_data=[X[valid], Y[valid]])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 257)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 1024)        264192    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, None, 1024)        0         
_________________________________________________________________
dense_2 (Dense)              (None, None, 1024)        1049600   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, None, 1024)        0         
_________________________________________________________________
dense_3 (Dense)              (None, None, 1024)        1049600   
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, None, 1024)        0         
__________

In [8]:
np.abs(Y).mean() # so learnability is poor, as we have 20% error...

0.11072513

0.1414213562373095

In [ ]:
Y[:1]

In [5]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

1.1857142857142857

In [6]:
pr = predict.predict(X[:160])
pr = K.ctc_decode(pr, X_lens[:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.3,
 0.23255813953488372,
 0.45161290322580644,
 0.275,
 0.4297520661157025,
 0.2127659574468085,
 0.24074074074074073,
 0.17777777777777778,
 0.25,
 0.23478260869565218,
 0.24050632911392406,
 0.19444444444444445,
 0.16923076923076924,
 0.5441176470588235,
 0.3333333333333333,
 0.4852941176470588,
 0.27631578947368424,
 0.2926829268292683,
 0.19101123595505617,
 0.3253012048192771,
 0.16071428571428573,
 0.18604651162790697,
 0.1958762886597938,
 0.29411764705882354,
 0.23076923076923078,
 0.3564356435643564,
 0.19480519480519481,
 0.25757575757575757,
 0.2777777777777778,
 0.21212121212121213,
 0.1797752808988764,
 0.2222222222222222,
 0.20987654320987653,
 0.3283582089552239,
 0.28125,
 0.24358974358974358,
 0.4939759036144578,
 0.24050632911392406,
 0.23255813953488372,
 0.19540229885057472,
 0.19753086419753085,
 0.19047619047619047,
 0.13253012048192772,
 0.367816091954023,
 0.24175824175824176,
 0.30434782608695654,
 0.43661971830985913,
 0.27848101265822783,
 0.20270270270270

In [7]:
np.array(_).mean()

0.2719325210045632